# Install and Import Packages

In [2]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\tpeiq\appdata\local\temp\pip-req-build-3kyplnod
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 74f9fe43758b5e5b79b2f269ce80bb0a54e72582
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\tpeiq\appdata\local\temp\pip-install-7py3xcrh\elegantrl_994fe6f5969849dd9671ccd7662606bb
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 8ea76afc3e7f1564ae9f0e69e70254116d575fe9
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-req-build-3kyplnod'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-install-7py3xcrh\elegantrl_994fe6f5969849dd9671ccd7662606bb'


In [3]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
import itertools
check_and_make_directories([TRAINED_MODEL_DIR])



In [4]:
from datetime import datetime, timedelta

# Set the end date as today
TRADE_END_DATE = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# Set the trade start date as 3 months before the end date
TRADE_START_DATE = (datetime.today() - timedelta(days=90)).strftime('%Y-%m-%d')

# Set the train end date as the trade start date
TRAIN_END_DATE = TRADE_START_DATE

# Set the train start date as 5 years before the train end date
TRAIN_START_DATE = (datetime.strptime(TRAIN_END_DATE, '%Y-%m-%d') - timedelta(days=5*365)).strftime('%Y-%m-%d')

print("TRAIN_START_DATE:", TRAIN_START_DATE)
print("TRAIN_END_DATE:", TRAIN_END_DATE)
print("TRADE_START_DATE:", TRADE_START_DATE)
print("TRADE_END_DATE:", TRADE_END_DATE)

TRAIN_START_DATE: 2020-03-28
TRAIN_END_DATE: 2025-03-27
TRADE_START_DATE: 2025-03-27
TRADE_END_DATE: 2025-06-24


In [5]:
symbols = ['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v']

In [6]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = symbols).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (22355, 8)


In [7]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2020-03-30,61.756058,61.928136,60.444883,60.769650,167976400,aapl,0
1,2020-03-30,47.860001,48.459999,46.660000,47.240002,68486600,amd,0
2,2020-03-30,98.197502,98.681503,95.616997,96.141502,122522000,amzn,0
3,2020-03-30,99.949753,100.960790,93.883517,95.136134,5725600,cat,0
4,2020-03-30,57.720001,59.830002,56.270000,58.810001,4449500,crwd,0


# Preprocess Data

In [8]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (1314, 8)
Successfully added vix


Successfully added turbulence index


In [9]:
processed.tic.unique()

array(['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v'],
      dtype=object)

In [10]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [11]:
processed_full.tic.unique()

array(['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v'],
      dtype=object)

# Save the Data

In [12]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

21335
1003


In [13]:
train_path = './data/train_data.csv'
trade_path = './data/trade_data.csv'

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)

# Load the Data from Part 1

In [14]:
train = pd.read_csv(train_path)

train = train.set_index(train.columns[0])
train.index.names = ['']

## Setup Environment

In [15]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 17, State Space: 171


In [16]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [17]:
train

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2020-03-30,aapl,61.756058,61.928136,60.444883,60.769650,167976400.0,0.0,0.000000,61.871269,61.514824,0.000000,66.666667,100.000000,61.756058,61.756058,57.080002,0.00000
0,2020-03-30,amd,47.860001,48.459999,46.660000,47.240002,68486600.0,0.0,0.000000,61.871269,61.514824,0.000000,66.666667,100.000000,47.860001,47.860001,57.080002,0.00000
0,2020-03-30,amzn,98.197502,98.681503,95.616997,96.141502,122522000.0,0.0,0.000000,61.871269,61.514824,0.000000,66.666667,100.000000,98.197502,98.197502,57.080002,0.00000
0,2020-03-30,cat,99.949753,100.960790,93.883517,95.136134,5725600.0,0.0,0.000000,61.871269,61.514824,0.000000,66.666667,100.000000,99.949753,99.949753,57.080002,0.00000
0,2020-03-30,crwd,57.720001,59.830002,56.270000,58.810001,4449500.0,0.0,0.000000,61.871269,61.514824,0.000000,66.666667,100.000000,57.720001,57.720001,57.080002,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2025-03-26,nvda,113.752106,118.831747,112.702176,118.721762,293463300.0,2.0,-2.411499,125.447755,107.424587,44.591860,-61.604423,21.469801,122.504869,128.046151,18.330000,6.79694
1254,2025-03-26,pypl,69.970001,71.620003,69.544998,70.930000,7449600.0,2.0,-1.720825,71.826010,66.961993,41.654447,-28.779887,13.777547,71.654667,78.866084,18.330000,6.79694
1254,2025-03-26,t,27.359606,27.547611,27.072652,27.122127,42589400.0,2.0,0.457903,27.603389,25.430711,63.638708,150.102749,28.315459,26.316347,24.574912,18.330000,6.79694


### Environment for Training

In [18]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Train Agent

In [19]:
agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

## A2C Model

The code above and below provide a healthy framework to add different agent types for training within the same notebook. For this I'll only be training an a2c model, but may add ddpg, ppo, and td3 in later iterations.

In [20]:
model_a2c = agent.get_model("a2c")
model_ppo = agent.get_model('ppo')

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cuda device
Logging to results/a2c
Logging to results/ppo


c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'`

In [21]:
import torch
print(torch.cuda.is_available())
if torch.cuda.is_available():
	print(torch.cuda.get_device_name(0))
else:
	print("CUDA is not available or PyTorch is not compiled with CUDA support.")
	print(torch.__version__)

True
NVIDIA GeForce RTX 3070 Laptop GPU


In [22]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

---------------------------------------
| time/                 |             |
|    fps                | 146         |
|    iterations         | 100         |
|    time_elapsed       | 3           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -24.1       |
|    explained_variance | -0.492      |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | 166         |
|    reward             | -0.59482515 |
|    reward_max         | 5.3874035   |
|    reward_mean        | 2.4613779   |
|    reward_min         | -0.59482515 |
|    std                | 0.999       |
|    value_loss         | 54.8        |
---------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 159        |
|    iterations         | 200        |
|    time_elapsed       | 6          |
|    total_timesteps    | 1000       |
| trai

In [23]:
trained_a2c.save(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None
trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo") if if_using_ppo else None

In [24]:

if_using_sac = True

model_sac = agent.get_model("sac")


if if_using_sac:
    tmp_path = RESULTS_DIR + '/sac'
    # Only use basic loggers to avoid rollout_buffer errors
    new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    model_sac.set_logger(new_logger_sac)





{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device
Logging to results/sac


In [25]:
trained_sac = agent.train_model(model=model_sac, tb_log_name='sac', total_timesteps=50000) if if_using_sac else None

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

In [26]:
trained_sac.save(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None